# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hamilton,39.1834,-84.5333,24.89,53,0,2.57,US,1686407272
1,1,blackmans bay,-43.0167,147.3167,10.41,76,51,3.28,AU,1686407359
2,2,manokwari,-0.8667,134.0833,26.42,81,17,1.12,ID,1686407359
3,3,st. john's,47.5649,-52.7093,8.71,97,100,6.69,CA,1686407162
4,4,edinburgh of the seven seas,-37.0676,-12.3116,14.08,72,81,9.13,SH,1686407359


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values

combined_data = city_data_df.loc[(city_data_df["Max Temp"]< 27 ) &  (city_data_df["Max Temp"] > 21) & \
                (city_data_df["Wind Speed"]< 4.5 ) & (city_data_df["Cloudiness"]== 0) ]

# Drop any rows with null values

#combined_data = combined_data.dropna()


# Display sample data
combined_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hamilton,39.1834,-84.5333,24.89,53,0,2.57,US,1686407272
49,49,stanley,54.8680,-1.6985,22.63,68,0,4.12,GB,1686407071
86,86,tsiombe,-25.3000,45.4833,23.45,62,0,3.29,MG,1686407370
119,119,camapua,-19.5314,-54.0439,26.78,42,0,2.44,BR,1686407374
127,127,jamestown,42.0970,-79.2353,22.23,60,0,2.57,US,1686407118


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity


hotel_df = combined_data[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hamilton,US,39.1834,-84.5333,53,
49,stanley,GB,54.8680,-1.6985,68,
86,tsiombe,MG,-25.3000,45.4833,62,
119,camapua,BR,-19.5314,-54.0439,42,
127,jamestown,US,42.0970,-79.2353,60,
201,fort bragg,US,35.1390,-79.0060,40,
220,howell,US,42.6073,-83.9294,49,
237,ambodifotatra,MG,-16.9833,49.8500,66,
288,darwin,AU,-12.4611,130.8418,78,
289,toliara,MG,-23.3500,43.6667,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
     print(row)
     break 

City          hamilton
Country             US
Lat            39.1834
Lng           -84.5333
Humidity            53
Hotel Name            
Name: 0, dtype: object


In [20]:
import pprint

# Set parameters to search for a hotel

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
      
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
   
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
   # pprint.pprint( name_address)
   # break       
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
stanley - nearest hotel: Hotel 52
tsiombe - nearest hotel: No hotel found
camapua - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
fort bragg - nearest hotel: Airborne Inn Lodging
howell - nearest hotel: Holiday Inn Express Howell
ambodifotatra - nearest hotel: Les Palmiers
darwin - nearest hotel: Mantra Pantanas Darwin
toliara - nearest hotel: Ambary
holualoa - nearest hotel: Kona Hotel
araxa - nearest hotel: Morada do Sol
metropolis - nearest hotel: Baymont Inn & Suites Metropolis
gordon - nearest hotel: Fairfield Inn & Suites Calhoun
ostersund - nearest hotel: Hotell Stortorget
maneromango - nearest hotel: No hotel found
ankazoabo - nearest hotel: No hotel found
spartanburg - nearest hotel: AC Hotel Spartanburg


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hamilton,US,39.1834,-84.5333,53,North Vista Manor
49,stanley,GB,54.8680,-1.6985,68,Hotel 52
86,tsiombe,MG,-25.3000,45.4833,62,No hotel found
119,camapua,BR,-19.5314,-54.0439,42,No hotel found
127,jamestown,US,42.0970,-79.2353,60,DoubleTree Jamestown
201,fort bragg,US,35.1390,-79.0060,40,Airborne Inn Lodging
220,howell,US,42.6073,-83.9294,49,Holiday Inn Express Howell
237,ambodifotatra,MG,-16.9833,49.8500,66,Les Palmiers
288,darwin,AU,-12.4611,130.8418,78,Mantra Pantanas Darwin
289,toliara,MG,-23.3500,43.6667,66,Ambary


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)